In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from gsd_utils import evaluate_segmentation
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    confusion_matrix,
    precision_recall_fscore_support,
    precision_score,
    recall_score,
    f1_score,
)
from pycm import ConfusionMatrix
import json
import cv2
# plt.rcParams.update({'font.size': 4})

data_path = Path("/data/M2F_pred_finetuning")
pred_path = data_path / "pred/output_test"
true_path = data_path / "target/Test_Annotated_masks"

preds_dir = Path("predictions")
m2fpreds_dir = preds_dir / "M2F-FT"
m2fpreds_dir.mkdir(exist_ok=True, parents=True)

confmat_dir = preds_dir / "confmat"
confmat_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
ious, accs, f1s, all_predictions, all_annotations = evaluate_segmentation(
    pred_path,
    true_path,
    ignored_classes=[1],
    num_classes=26,
)
print(all_predictions.shape, all_annotations.shape)
np.save(m2fpreds_dir / "predictions.npy", all_predictions)
np.save(m2fpreds_dir / "annotations.npy", all_annotations)

In [ ]:
class_mapping = json.loads(Path("label_to_id.json").read_bytes())
id_class = {y: x for x, y in class_mapping.items()}

In [ ]:
pred_paths = [p for p in pred_path.rglob("*.png")]
file_map = {pred: next(true_path.rglob(f"{pred.stem}*")) for pred in pred_paths}

pred_arrs = [cv2.imread(p, cv2.IMREAD_GRAYSCALE) for p in file_map.keys()]
true_arrs = [cv2.imread(file_map[p], cv2.IMREAD_GRAYSCALE) for p in file_map.keys()]

pred_values = np.vstack([im.flatten() for im in pred_arrs]).flatten()
true_values = np.vstack([im.flatten() for im in true_arrs]).flatten()

pred_labels = np.vectorize(id_class.get)(pred_values)
true_labels = np.vectorize(id_class.get)(true_values)

all_labels = np.vectorize(id_class.get)(np.arange(len(id_class)))

pred_labels.shape, true_labels.shape

In [ ]:
all_model_values = np.stack([pred_values, true_values])
all_uniques = np.unique(all_model_values)
present_labels = [id_class[idx] for idx in all_uniques]
inconsistent = [v for v in np.arange(len(id_class)) if v not in all_uniques]
inconsistent, [id_class[cl] for cl in inconsistent]

In [ ]:
classification = {"target": true_values, "pred": pred_values}
precs = precision_score(classification["target"], classification["pred"], average=None)
f1s = f1_score(classification["target"], classification["pred"], average=None)
recs = recall_score(classification["target"], classification["pred"], average=None)

In [ ]:
metrics = pd.DataFrame(
    {"name": present_labels, "f1": f1s, "precision": precs, "recall": recs}
)
metrics.precision *= 100
metrics.f1 *= 100
metrics.recall *= 100
metrics

In [ ]:
metrics.to_csv(m2fpreds_dir / "metrics.csv", index=False)

In [ ]:
classification = {"target": all_annotations, "pred": all_predictions}
precs = precision_score(classification["target"], classification["pred"], average=None)
f1s = f1_score(classification["target"], classification["pred"], average=None)
recs = recall_score(classification["target"], classification["pred"], average=None)
class_metrics = pd.DataFrame(
    {"name": present_labels, "f1": f1s, "precision": precs, "recall": recs}
)
class_metrics.precision *= 100
class_metrics.f1 *= 100
class_metrics.recall *= 100
class_metrics

In [ ]:
cm = ConfusionMatrix(all_annotations, all_predictions, is_imbalanced=True)
cm.plot(normalized=True, cmap="Blues")
cm.save_html(str(confmat_dir / "them"))

In [ ]:
cm = ConfusionMatrix(
    actual_vector=true_labels,
    predict_vector=pred_labels,
    is_imbalanced=True,
)
cm.plot(normalized=True, cmap="Blues", number_label=False)
cm.save_html(str(confmat_dir / "labelled"), color="Blues", normalize=True)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
disp = ConfusionMatrixDisplay.from_predictions(
    all_annotations,
    all_predictions,
    cmap="Blues",
    normalize="true",
    values_format="2%",
    ax=ax,
)

In [ ]:
cm = confusion_matrix(true_values, pred_values, normalize="true")
cmp = ConfusionMatrixDisplay(cm, display_labels=[id_class[cl] for cl in all_uniques])
fig, ax = plt.subplots(figsize=(30, 10))
cmp.plot(
    ax=ax,
    colorbar=False,
    cmap="Blues",
    values_format="2.0%",
    include_values=False,
)
cax = fig.add_axes(
    [ax.get_position().x1 + 0.01, ax.get_position().y0, 0.02, ax.get_position().height]
)
plt.colorbar(cmp.im_, cax=cax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
disp = ConfusionMatrixDisplay.from_predictions(
    true_labels,
    pred_labels,
    cmap="Blues",
    normalize="true",
    values_format="%",
    ax=ax,
)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")

In [ ]:
values = {
    "them": {"target": all_annotations, "pred": all_predictions},
    "ours": {"target": true_values, "pred": pred_values},
}
# Precision
for k, v in values.items():
    print(
        k,
        precision_score(v["target"], v["pred"], average="weighted"),
        recall_score(v["target"], v["pred"], average="weighted"),
        f1_score(v["target"], v["pred"], average="weighted"),
    )

In [ ]:
f1_score(
    classification["target"],
    classification["pred"],
    average=None,
    labels=[str(a) for a in np.arange(24)],
)

In [ ]:
cm = confusion_matrix(true_values, pred_values)
cmp = ConfusionMatrixDisplay(cm, display_labels=all_labels)
cm